In [1]:
import sys
import pandas as pd
sys.path.insert(0,'../')
from nps_model_functions import *
from environment import env
config = env.env()

In [2]:
# Amenities
query = 'List the amenities at Zion'
endpoint, parkcode, intent = nps_model_functions.get_params(query)
print(endpoint, parkcode, intent)


amenities zion amenities
